# Домашняя работа 4 · Нейросетевые модели временных рядов

## План ноутбука
1. Загрузка данных и подготовка
2. Базовое решение с MLP
3. Улучшенное решение с RNN
4. Foundational models
5. Финальный пайплайн для Kaggle

Документация по нейросетям в ETNA: https://docs.etna.ai/stable/tutorials/202-NN_examples.html

In [ ]:
import os
import json
import warnings
import logging

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import random

warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
HORIZON = 366
SEGMENT_LIMIT = 25
DATA_PATH = "https://gist.githubusercontent.com/martins0n/86ca7e632a57bb1faefa98dc8e467d17/raw/2dfa5e6acf97ae2f6a587fc5e1c629836a8af16b/train.parquet"

In [ ]:
from etna.metrics import SMAPE
from etna.pipeline import Pipeline
from etna.datasets import TSDataset
from etna.analysis import plot_backtest


METRIC = SMAPE()

def evaluate_pipeline(pipeline: Pipeline, ts: TSDataset) -> float:
    '''Запускаем один фолд backtest и возвращаем средний SMAPE по всем сегментам.'''
    backtest_output = pipeline.backtest(ts=ts, metrics=[METRIC], n_folds=1)
    fold_mean = backtest_output["metrics"].groupby("fold_number")["SMAPE"].mean().iloc[0]
    plot_backtest(backtest_output["forecasts"], ts, history_len=HORIZON*2)
    return fold_mean

## 1. Загрузка данных и sanity-check
**Что сделать:** 
- Здесь ничего не нужно делать =)

In [ ]:
from etna.datasets import TSDataset

raw_df = pd.read_parquet(DATA_PATH)
unique_segments = raw_df["segment"].nunique()

if SEGMENT_LIMIT is not None:
    segments_to_keep = raw_df["segment"].unique()[:SEGMENT_LIMIT]
    raw_df = raw_df[raw_df["segment"].isin(segments_to_keep)]

ts = TSDataset(df=TSDataset.to_dataset(raw_df), freq="D")

In [ ]:
# Не редактируй этот блок кода, он нужен для автоматической проверки

summary = {
    "n_segments": len(ts.segments),
}
print(json.dumps(summary, indent=2, ensure_ascii=False))

In [ ]:
ts.plot(n_segments=4)

## 2. Бейзлайн: MLP
**Что сделать:**
- Построить бейзлайн с помощью MLP

In [ ]:
set_seed()

In [ ]:
from etna.models.nn import MLPModel
from etna.transforms import StandardScalerTransform
from etna.transforms import LagTransform
from etna.transforms import DateFlagsTransform
from etna.transforms import LabelEncoderTransform
from etna.transforms import SegmentEncoderTransform


# TODO: Соберите необходимые трансформации для MLP модели
# Посмотрите в сторону лагов, datetime фич и кодирования сегментов

mlp_transforms = [
   # Ваши трансформации
]
mlp_model = MLPModel(
    decoder_length=HORIZON,
    # Ваши параметры модели
)

mlp_pipeline = Pipeline(
    model=mlp_model,
    transforms=mlp_transforms,
    horizon=HORIZON,
)


mlp_smape = evaluate_pipeline(mlp_pipeline, ts)

In [ ]:
# Не редактируй этот блок кода, он нужен для автоматической проверки

result = {
        "mlp_smape": mlp_smape, # < 23
}
print(json.dumps(result, indent=2, ensure_ascii=False))

## 3. RNN vs RNN: дефолтная версия и улучшенная
**Что сделать:**
- Построить два решения с RNN: дефолтное (без признаков) и улучшенное

In [ ]:
set_seed()

In [ ]:
from etna.models.nn import RNNModel

rnn_baseline_transforms = [
    StandardScalerTransform(in_column="target"),
]

#TODO: Настройте RNN модель с дефолтными параметрами (без фич)


rnn_baseline_model = RNNModel(
    decoder_length=HORIZON,
    # Ваши параметры модели
)

rnn_baseline_pipeline = Pipeline(
    model=rnn_baseline_model,
    horizon=HORIZON,
    transforms=rnn_baseline_transforms,
)

rnn_baseline_smape = evaluate_pipeline(rnn_baseline_pipeline, ts=ts)

In [ ]:
set_seed()

In [ ]:
from etna.models.nn import RNNModel


# TODO: Улучшите базовую RNN модель


rnn_modified_transforms = [
    # Ваши трансформации
]


rnn_modified_model = RNNModel(
    # Ваши параметры модели
)


rnn_modified_pipeline = Pipeline(
    model=rnn_modified_model,
    horizon=HORIZON,
    transforms=rnn_modified_transforms,
)

rnn_modified_smape = evaluate_pipeline(rnn_modified_pipeline, ts=ts)

In [ ]:
# Не редактируй этот блок кода, он нужен для автоматической проверки

result = {
    "rnn_baseline_smape": rnn_baseline_smape,  # < 30
    "rnn_modified_smape": rnn_modified_smape,  # < 25
    "compared": bool(rnn_modified_smape < rnn_baseline_smape)
}
print(json.dumps(result, indent=2, ensure_ascii=False))

## 4. Foundational model: Chronos
**Что сделать:**


In [ ]:
set_seed()

In [ ]:
from etna.models.nn import ChronosBoltModel


chronos_model = ChronosBoltModel(
    # Ваши параметры модели
)

chronos_pipeline = Pipeline(
    model=chronos_model,
    horizon=HORIZON,
)

chronos_smape = evaluate_pipeline(chronos_pipeline, ts=ts)

In [ ]:
# Не редактируй этот блок кода, он нужен для автоматической проверки

result = {
    "chronos_smape": chronos_smape,  # < 25
}
print(json.dumps(result, indent=2, ensure_ascii=False))

## 5. Сделайте пайплайн для финального прогноза для kaggle
**Что сделать:**
- Выберите ваш пайплайн
- Здесь можно экспериментировать с разными стратегиями агрегации прогнозов по сегментам и т.д.
- Запускать backtest в этом задании не нужно, но сделайте это для себя локально, чтобы убедиться, что ваш пайплайн работает лучше других

In [ ]:
import json
from etna.pipeline import Pipeline

# TODO: Соберите пайплайн для финального прогноза на тестовый период для kaggle
model = ...
transforms = ...

pipe = ...


In [ ]:
# Не редактируй этот блок кода, он нужен для автоматической проверки

result = {
    "сurrent_pipeline": pipe.horizon
}

print(json.dumps(result))